#### Ejemplos con BertTokenizer

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from transformers import BertTokenizer
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import random
from collections import Counter
from datasets import load_dataset
import pandas as pd

/Users/dac/miniconda3/envs/torchenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from funciones_dataset import ETIQUETAS_PUNT_INICIAL, ETIQUETAS_PUNT_FINAL, ETIQUETAS_CAPITALIZACION

In [6]:
#pip install -U datasets

In [7]:
url = 'https://raw.githubusercontent.com/LeoBrasileo/Aprendizaje-Automatico-TP2/main/datos_google_argento.csv'
df_google_arg = pd.read_csv(url, sep=';', index_col=False)
df_google_arg.head()

,texto_original,texto_limpio
0,"Para la caída del cabello, tengo un nuevo champú.",para la caída del cabello tengo un nuevo champú
1,Los hámsters comen zanahorias.,los hámsters comen zanahorias
2,¿Me podés mandar fotos de la pileta?,me podés mandar fotos de la pileta
3,Hola Cristina que bueno que pueda hablar con vos.,hola cristina que bueno que pueda hablar con vos
4,Tranquilo va a estar todo bien.,tranquilo va a estar todo bien


In [8]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

original = '¿A qué hora vamos a McDonalds?'
texto = "a qué hora vamos a mcdonalds"
tokens = tokenizer.tokenize(texto)
tokens_orig = tokenizer.tokenize(original)

print(tokens)
print(tokens_orig)

['a', 'qué', 'hora', 'va', '##mos', 'a', 'm', '##c', '##dona', '##ld', '##s']
['¿', 'A', 'qué', 'hora', 'va', '##mos', 'a', 'McDonald', '##s', '?']


In [9]:
print(tokenizer.convert_tokens_to_ids(tokens))

[169, 38188, 24301, 10321, 13386, 169, 181, 10350, 64674, 12620, 10107]


#### Limpieza de strings

In [10]:
vocales_con_acento_min = [chr(x) for x in [ord('á'), ord('é'), ord('í'), ord('ó'), ord('ú')]]
vocales_con_acento_may = [chr(x) for x in [ord('Á'), ord('É'), ord('Í'), ord('Ó'), ord('Ú')]]
allowed_characters = [chr(x) for x in range(97,123)] + vocales_con_acento_min + [chr(241)] + [chr(10), chr(32)] + [chr(x) for x in range(48,58)] # los caracteres de letras minúsculas, espacio en blanco y fin de linea y números
characters_to_replace = [chr(x) for x in range(65,91)] + vocales_con_acento_may + [chr(209)] #las mayúsculas
characters_to_replace_with = [chr(x) for x in range(97,123)] + vocales_con_acento_min + [chr(241)] #las minúsculas
replace_dict = dict(zip(characters_to_replace, characters_to_replace_with))

def limpiar_string (s) :
  res = ""
  for c in s :
    if c in allowed_characters :
      res += c
    elif c in characters_to_replace :
      res += replace_dict[c]
  return res

In [11]:
test_limpieza = "Veamos que pasa si cortamos un árbol ¿Será capaz de entender? El Árbol no sentirá el corte hasta que sea demaisado tarde: al buscar sus raices para nutrir las hojas, sentirá la desesperación de la falta." + '\n' + '¡Ósado el talador, que se preocupa más por el calor de su morada que la protección que ofrece el bosque y la protección de todas sus criaturas contra el silencio verdadero!'
print(test_limpieza)
print('\n')
print(limpiar_string(test_limpieza))

Veamos que pasa si cortamos un árbol ¿Será capaz de entender? El Árbol no sentirá el corte hasta que sea demaisado tarde: al buscar sus raices para nutrir las hojas, sentirá la desesperación de la falta.
¡Ósado el talador, que se preocupa más por el calor de su morada que la protección que ofrece el bosque y la protección de todas sus criaturas contra el silencio verdadero!


veamos que pasa si cortamos un árbol será capaz de entender el árbol no sentirá el corte hasta que sea demaisado tarde al buscar sus raices para nutrir las hojas sentirá la desesperación de la falta
ósado el talador que se preocupa más por el calor de su morada que la protección que ofrece el bosque y la protección de todas sus criaturas contra el silencio verdadero


#### Etiquetas para datos de entrada

**Nota**: es un planteo preliminar, imitando cómo nos piden entregar las respuestas, se pueden hacer modificaciones para lo que necesitemos como entrada

In [12]:
import pandas as pd
from funciones_dataset import generar_data_palabras, asignar_etiquetas_puntuacion, asignar_puntuacion_a_tokens

Ejemplo 1:

Primero definimos puntuación para los tokens

In [18]:
texto_original = "¿Cuándo vamos a McDonald's?Ellos no vienen. Entonces, - ¿dónde están?"

texto_limpio = limpiar_string(texto_original)
tokens_texto_limpio = tokenizer.tokenize(texto_limpio)

puntuacion_tokens = asignar_puntuacion_a_tokens(instancia_original=texto_original,
                                                instancia_id=1,
                                                instancia_tokens=tokens_texto_limpio)
pd.DataFrame.from_dict(puntuacion_tokens)

,instancia_id,token,token_id,capitalizacion,puntuacion_inicial,puntuacion_final
0,1,cu,10854,1,¿,
1,1,##ánd,101439,1,,
2,1,##o,10133,1,,
3,1,va,10321,0,,
4,1,##mos,13386,0,,
5,1,a,169,0,,
6,1,m,181,2,,
7,1,##c,10350,2,,
8,1,##dona,64674,2,,
9,1,##ld,12620,2,,


La instancia etiquetada final (con las etiquetas definidas en _ETIQUETA_PUNT_FINAL_ y _ETIQUETA_PUNT_INICIAL_) sería de esta forma:

In [31]:
tokens_etiquetados = asignar_etiquetas_puntuacion(puntuacion_tokens)
pd.DataFrame.from_dict(tokens_etiquetados)

,instancia_id,token,token_id,capitalizacion,puntuacion_inicial,puntuacion_final
0,1,cu,10854,1,1,0
1,1,##ánd,101439,1,0,0
2,1,##o,10133,1,0,0
3,1,va,10321,0,0,0
4,1,##mos,13386,0,0,0
5,1,a,169,0,0,0
6,1,m,181,2,0,0
7,1,##c,10350,2,0,0
8,1,##dona,64674,2,0,0
9,1,##ld,12620,2,0,0


Ejemplo 2:

In [36]:
texto_original = "¿Cuándo vamos a McDonald's? Ellos no vienen. Entonces, ¿dónde están?"

texto_limpio = limpiar_string(texto_original)
tokens_texto_limpio = tokenizer.tokenize(texto_limpio)

puntuacion_tokens = asignar_puntuacion_a_tokens(instancia_original=texto_original,
                                                instancia_id=1,
                                                instancia_tokens=tokens_texto_limpio)

tokens_etiquetados = asignar_etiquetas_puntuacion(puntuacion_tokens)

pd.DataFrame.from_dict(tokens_etiquetados)

,instancia_id,token,token_id,capitalizacion,puntuacion_inicial,puntuacion_final
0,1,cu,10854,1,1,0
1,1,##ánd,101439,1,0,0
2,1,##o,10133,1,0,0
3,1,va,10321,0,0,0
4,1,##mos,13386,0,0,0
5,1,a,169,0,0,0
6,1,m,181,2,0,0
7,1,##c,10350,2,0,0
8,1,##dona,64674,2,0,0
9,1,##ld,12620,2,0,0


In [21]:
tokens_test_limpieza = tokenizer.tokenize(limpiar_string(test_limpieza))

puntuacion_tokens = asignar_puntuacion_a_tokens(instancia_original=test_limpieza,
                                                instancia_id=1,
                                                instancia_tokens=tokens_test_limpieza)

tokens_etiquetados = asignar_etiquetas_puntuacion(puntuacion_tokens)

pd.DataFrame.from_dict(tokens_etiquetados).head(12)

,instancia_id,token,token_id,capitalizacion,puntuacion_inicial,puntuacion_final
0,1,ve,10323,1,0,0
1,1,##amos,73983,1,0,0
2,1,que,10121,0,0,0
3,1,pasa,26088,0,0,0
4,1,si,10294,0,0,0
5,1,corta,53134,0,0,0
6,1,##mos,13386,0,0,0
7,1,un,10119,0,0,0
8,1,árbol,55220,0,0,0
9,1,será,23843,1,1,0


In [19]:
#creaar DataFrame para alimentar el dataset (pensando en que va a ser mas rapido para el entrenamiento)
data_path ='../data/marquez_cleaned.csv'  # Remove the leading slash
data = pd.read_csv(data_path, delimiter=',')
texto_original = data['texto_original'].tolist()
# texto_limpio = data['texto_limpio'].tolist()

puntuacion_tokens_marquez = []
problematic_sentences = []  

for i in range(4,len(texto_original)):
    try:
        # tokens_texto_original = tokenizer.tokenize(texto_original[i])
        texto_limpio = limpiar_string(texto_original[i])
        tokens_texto_limpio = tokenizer.tokenize(texto_limpio)

        puntuacion_tokens = asignar_puntuacion_a_tokens(instancia_original=texto_original[i],
                                                        instancia_id=i+1,
                                                        instancia_tokens=tokens_texto_limpio)
        puntuacion_tokens = asignar_etiquetas_puntuacion(puntuacion_tokens)
        puntuacion_tokens_marquez.append(puntuacion_tokens)
    except Exception as e:
        # Store problematic sentence info instead of breaking
        problematic_sentences.append({
            'index': i,
            'original_text': texto_original[i],
            'cleaned_text': texto_limpio if 'texto_limpio' in locals() else None,
            'error': str(e)
        })
        continue  


print(f"funcionaron: {len(puntuacion_tokens_marquez)} sentences")
print(f"con problemas: {len(problematic_sentences)}")

# oraciones con problemas
for prob in problematic_sentences[:5]:  # Show first 5 problematic sentences
    print(f"Index {prob['index']}: {repr(prob['original_text'][:100])}... Error: {prob['error']}")


# renumerar para que los ids de instancia sean consecutivos
puntuacion_tokens_marquez_clean = []
for new_id, tokens_list in enumerate(puntuacion_tokens_marquez):
    tokens_list_updated = []
    for token_dict in tokens_list:
        token_dict_updated = token_dict.copy()
        token_dict_updated['instancia_id'] = new_id + 1  # que empiece en 1?
        tokens_list_updated.append(token_dict_updated)
    puntuacion_tokens_marquez_clean.append(tokens_list_updated)


funcionaron: 2004 sentences
con problemas: 142
Index 8: 'Dentro de poco, el hombre podrá ver lo que ocurre en cualquier lugar de la tierra, sin moverse de su'... Error: string index out of range
Index 14: 'Durante varios años esperó la respuesta. Por último, cansado de esperar, se lamentó ante Rubén del f'... Error: string index out of range
Index 21: 'Toda la aldea estaba convencida de que Alejandra Buendia había perdido el juicio, cuando llegó Javie'... Error: string index out of range
Index 62: 'Sólo cuando empezó a desmontar la puerta del cuartito, Ángel se atrevió a preguntarle por qué lo hac'... Error: string index out of range
Index 64: '- Todavía no tenemos un muerto - dijo él -. Uno no es de ninguna parte mientras no tenga un muerto b'... Error: string index out of range


In [21]:
class MarquezDataset(Dataset):
    def __init__(self, data, label_key='puntuacion_final', pad_token_id=0, pad_label_id=-100):
        """
        Args:
            data: list of sentences, each a list of dicts (one per token)
            label_key: which label to use ('puntuacion_final' or 'puntuacion_inicial')
            pad_token_id: ID used for padding tokens
            pad_label_id: label used for padding positions (-100 for CrossEntropyLoss)
        """
        self.data = data
        self.label_key = label_key
        self.pad_token_id = pad_token_id
        self.pad_label_id = pad_label_id

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sentence = self.data[idx]

        # Extract features
        token_ids = [tok['token_id'] for tok in sentence]
        caps = [tok['capitalizacion'] for tok in sentence]
        punt_inicial = [tok['puntuacion_inicial'] for tok in sentence]
        punt_final = [tok[self.label_key] for tok in sentence]
        
        return {
            'token_ids': torch.tensor(token_ids, dtype=torch.long),
            'capitalizacion': torch.tensor(caps, dtype=torch.float),
            'puntuacion_inicial': torch.tensor(punt_inicial, dtype=torch.float),
            'puntuacion_final': torch.tensor(punt_final, dtype=torch.float),
            'length': len(token_ids)
        }
        
def marquez_collate_fn(batch, pad_token_id=0, pad_label_id=-100):
    # Separate each field
    token_ids = [item['token_ids'] for item in batch]
    caps = [item['capitalizacion'] for item in batch]
    punt_inicial = [item['puntuacion_inicial'] for item in batch]
    punt_final = [item['puntuacion_final'] for item in batch]
    lengths = [item['length'] for item in batch]

    # Pad sequences
    padded_token_ids = pad_sequence(token_ids, batch_first=True, padding_value=pad_token_id)
    padded_caps = pad_sequence(caps, batch_first=True, padding_value=0.0)
    padded_punt_inicial = pad_sequence(punt_inicial, batch_first=True, padding_value=pad_label_id)
    padded_punt_final = pad_sequence(punt_final, batch_first=True, padding_value=pad_label_id)

    return {
        'token_ids': padded_token_ids,
        'capitalizacion': padded_caps,
        'puntuacion_inicial': padded_punt_inicial,
        'puntuacion_final': padded_punt_final,
        'lengths': torch.tensor(lengths)
    }
    
dataset = MarquezDataset(data=puntuacion_tokens_marquez)
loader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=marquez_collate_fn)
dataset[0]

{'token_ids': tensor([ 13436,  39738,  10228,  10104,  22755,  13859,  14386,  16002,  10220,
          10266,  16898,  13859,  10109,  12088,  76456,  25303,  10443,  10198,
          40407,  11047,  15518,  17359,  10126,  10266,  11355,  55168,  10110,
          77644,  10125,  13621,  93548,  10104,  10846,  10173,  18533,  35854,
          11419,  46128,  17671, 110761,  10133,    169, 110761,  10133,  10109,
          16284,  34379,  10125,  27335,  10127,  15048,  38192,  19975,  10605,
          10182,  10398,  11614,  32448,  11197,  10104,  83141,    193,  11639,
          32502,  10605,  10110,  19650,  14855,  10125,  10173,  56671,  10133,
          10104,  39215]),
 'capitalizacion': tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.